In [34]:
import MySQLdb

import pandas as pd

import re
import json

In [35]:
location = "127.0.0.1"

user = "root"

password = ""

database_name = "bigcookingdata"

db = None

cursor = None

def connect():
    global cursor
    global db
    db = MySQLdb.connect(location,user,password,database_name )
    cursor = db.cursor()
def disconnect():
    db.close()

def get_version():
    cursor.execute( "SELECT VERSION()" )
    return cursor.fetchone()[0]

In [36]:
# Create Databases
def createDB():
    connect()
    sql_create_db = """
        DROP TABLE IF EXISTS assess;
        DROP TABLE IF EXISTS client;
        DROP TABLE IF EXISTS contain_recipe_ingredient;
        DROP TABLE IF EXISTS have_preferences_ingredient;
        DROP TABLE IF EXISTS ingredient;
        DROP TABLE IF EXISTS recipe;
        DROP TABLE IF EXISTS record;


       CREATE TABLE assess (
        id_recipe int(11) NOT NULL,
        id_client int(11) NOT NULL,
        rating tinyint(4) DEFAULT NULL,
        commentary varchar(280) DEFAULT NULL,
        PRIMARY KEY (id_recipe,id_client),
        KEY id_client (id_client)
        ) ENGINE=MyISAM DEFAULT CHARSET=latin1
        ;

        CREATE TABLE client (
        id_client int(11) NOT NULL,
        firstname varchar(25) DEFAULT NULL,
        lastname varchar(25) DEFAULT NULL,
        civility char(2) DEFAULT NULL,
        pseudo varchar(25) DEFAULT NULL,
        mail varchar(40) DEFAULT NULL,
        password varchar(30) DEFAULT NULL,
        PRIMARY KEY (id_client)
        ) ENGINE=MyISAM DEFAULT CHARSET=latin1;

        CREATE TABLE have_preferences_ingredient (
        id_client int(11) NOT NULL,
        id_ingredient int(11) NOT NULL,
        PRIMARY KEY (id_client,id_ingredient),
        KEY id_ingredient (id_ingredient)
        ) ENGINE=MyISAM DEFAULT CHARSET=latin1;

        CREATE TABLE ingredient (
          id_ingredient int(11) NOT NULL,
          name varchar(40) DEFAULT NULL,
          url_pic text DEFAULT NULL,
          PRIMARY KEY (id_ingredient)
        ) ENGINE=MyISAM DEFAULT CHARSET=latin1;


        CREATE TABLE recipe (
          id_recipe int(11) NOT NULL,
          name varchar(50) DEFAULT NULL,
          categories text,
          url_pic text,
          clusterNumber varchar(20) NOT NULL,
          directions text,
          prep_time varchar(20) DEFAULT NULL,
          cook_time varchar(20) DEFAULT NULL,
          break_time varchar(20) DEFAULT NULL,
          difficulty varchar(15) DEFAULT NULL,
          budget varchar(15) DEFAULT NULL,
          serving varchar(6) DEFAULT NULL,
          coordonnees text DEFAULT NULL,
          PRIMARY KEY (id_recipe)
        ) ENGINE=MyISAM DEFAULT CHARSET=latin1;
        
        CREATE TABLE contain_recipe_ingredient (
          id_recipe int(11) NOT NULL,
          id_ingredient int(11) NOT NULL,
          quantity varchar(20) DEFAULT NULL,
          PRIMARY KEY (id_recipe,id_ingredient),
          KEY id_ingredient (id_ingredient)
        )ENGINE=MyISAM DEFAULT CHARSET=latin1;



        CREATE TABLE record (
          id_recipe int(11) NOT NULL,
          id_client int(11) NOT NULL,
          PRIMARY KEY (id_recipe,id_client),
          KEY id_client (id_client)
        ) ENGINE=MyISAM DEFAULT CHARSET=latin1;
    
    """
#     global sql_assess,sql_contain_recipe_ingredient,sql_have_preferences_ingredient,sql_ingredient
#     global sql_recipe,sql_client,sql_record,table_recipe,table_user
    sql_assess = "drop table assess;"
    sql_contain_recipe_ingredient = "drop table contain_recipe_ingredient;"
    sql_have_preferences_ingredient = "drop table have_preferences_ingredient;"
    sql_ingredient = "drop table ingredient;"
    sql_recipe = "drop table recipe;"
    sql_client = "drop table client;"
    sql_record = "drop table record;"
    table_recipe = 'l_recipe_ingredient'
    table_user = 'user'
    _SQL = """SHOW TABLES"""
    try:
#         cursor.execute( _SQL )
        cursor.execute( sql_create_db )
        results = cursor.fetchall()
        print( 'All existing tables:', results )  # Returned as a list of tuples
    except (MySQLdb.Error, MySQLdb.Warning) as e:
        print('error')
        print(e)
    results_list = [item[0] for item in results]  # Conversion to list of str
    if(table_recipe and table_user in results_list):
        print(table_recipe, table_record, table_user, "was found")
        try:
            # Execute the SQL command
            cursor.execute(sql_recipe)
            cursor.execute(sql_ingredient)
            cursor.execute(sql_ingredient)
            cursor.execute(sql_client)
            cursor.execute(sql_assess)
            cursor.execute(sql_contain_recipe_ingredient)
            cursor.execute(sql_have_preferences_ingredient)
            cursor.execute(sql_record)
            cursor.execute(sql_create_db)
            # Commit your changes in the database
            db.commit()
        except (MySQLdb.Error, MySQLdb.Warning) as e:
                db.rollback()
                print(e)
    else:
        print(table_recipe, table_user, "was found")
        try:
#             cursor.execute(sql_drop)
            cursor.execute( sql_create_db )
        except:
            db.rollback()
            disconnect()

            

In [37]:
createDB()

All existing tables: ()
l_recipe_ingredient user was found


In [38]:
def persisetDB(recipes):
    global sql_assess,sql_contain_recipe_have_preferences_ingredient,sql_ingredient,sql_recipe
    global sql_client,sql_record,  liste_ingre_totale_pd, categories, categorie_str, liste_ingre,dict_ingre
    global ing_id_unique, ing_quantity,list_utensil_total_pd,title_recipe,ingre, id_fetch,id_fetch_rec
    
    listDic_Ingre = []
    dict_ingre = {}
    connect()
    ################# Création table ingredients sans doublons #####################
    for values in recipes:
        ingredients = values['ingredients']
        categories =  values['category']
        title =  values['nom']
        img_url =  values['img_url']
        time_prepa =  values['time_prepa']
        time_repo =  values['time_repo']
        time_cuisson =  values['time_cuisson']
        difficulty =  values['difficulty']
        budget =  values['budget']
        directions =  "NO VALUES"
        if ingredients is None:
            pass
            print( "0 ingrédients" )
        else:
            for ing in ingredients:
                liste_ingre = []
                dict_Ingre = {}
                ing_id_unique = 0
                liste_ingre_by_recipe = []
                ing_name = re.escape( str( ing['nom_ingre']))
                ing_img_url = re.escape( str( ing['image_ingre']))
                dict_Ingre[ing_name] = ing_img_url
                listDic_Ingre.append(dict_Ingre)
               
                                        
#     print( "LEN LIST INGREDIENT :",listDic_Ingre)
    newListIngre = [i for n, i in enumerate(listDic_Ingre) if i not in listDic_Ingre[n + 1:]]
#     print( "LEN LIST INGREDIENT :",newListIngre)

    #insert ingredients in base
    counter = 0
    for key,ingredient in enumerate(newListIngre):
        key += 1
        for ingre, url_pic in ingredient.items():
            ingre = ingre.replace('\u0301', ' ')
            ingre = ingre.replace('\u00ae\ufe0f', ' ')
            sql_ingredient = """ insert into ingredient
            (name,id_ingredient,url_pic) values("%s","%s","%s");""" % (ingre, key,url_pic)
        print( "INGREDIENT :", sql_ingredient )

        try:
            # Execute the SQL command ingredient
            cursor.execute( sql_ingredient )
            # Commit your changes in the database
            db.commit()
        except:
            # Rollback in case there is any error
            db.rollback()
            
    
    # insert recette
    compteur = 0
   
    for values in recipes:
        name=  values.get('nom')
        name = name.replace("\"", "\\'")
        name = name.replace('\u0301', ' ')
        name = name.replace('\u00ae\ufe0f', ' ')
        prep_time =   values.get( 'time_prepa' )
        cook_time =   values.get( 'time_cuisson' )
        break_time =  values.get( 'time_repo' )
        rating = values.get( 'global_rating' )
        difficulty =  values.get( 'difficulty' )
        budget =  values.get( 'budget' )
        categories =  values.get( 'category' )
        categories = ';'.join(categories)
        categories = categories.replace(",", " ")
        
        title_recipe =  values.get( 'title' )
        directions =  "NO VALUES"
        url_pic =  values.get('img_url')

        clusterNumber = values.get('cluster')
        coordonnees = values.get('coordonnees')
        serving = 1

         # Affichage numerous recipe et compteur
        compteur = compteur + 1
#         print( "--- NUMEROUS RECIPE :", compteur, ":", name, "---" )

        # Condition if None everywhere
        if not name and not budget and not difficulty and not categories and not rating:
            name = 0
            prep_time =  0
            cook_time = 0
            break_time =0
            rating = 0
            difficulty = 0
            budget = 0
            categories = 0
            title_recipe =0
            utensils = 0
            etape =  0
            coordonnees = 0
            number_of_person = 0
            clusterNumber = 0.0

            sql_recipe = """insert into recipe (id_recipe,name,categories,url_pic,clusterNumber,directions,prep_time,cook_time,
            break_time,difficulty,budget,serving,coordonnees)\
                                        values ("%s","%s","%s","%s","%s","%s","%s","%s","%s","%s","%s","%s","%s");"""  % (
                    compteur,name, categories,url_pic, clusterNumber, directions, prep_time, cook_time, break_time,
                    difficulty,budget,serving,coordonnees)
            
      
        # Condition for Categories
        if categories is None:
            categories = 'Vide'
            print( "0 categorie" )
        else:
            pass
        
        if budget is None:
                budget = 0
                sql_recipe = """insert into recipe (id_recipe,name,categories,url_pic,clusterNumber,directions,prep_time,cook_time,
                                break_time,difficulty,budget,serving,coordonnees)\
                                values ("%s","%s","%s","%s","%s","%s","%s","%s","%s","%s","%s","%s","%s");"""  % (
                    compteur,name, categories, url_pic, clusterNumber, directions, prep_time, cook_time, break_time,
                    difficulty,budget,serving,coordonnees)
                
        elif rating is None:
                rating = 0
                sql_recipe = """insert into recipe (id_recipe,name,categories,url_pic,clusterNumber,directions,prep_time,cook_time,
                                break_time,difficulty,budget,serving,coordonnees)\
                                 values ("%s","%s","%s","%s","%s","%s","%s","%s","%s","%s","%s","%s","%s");""" % (
                    compteur,name, categories, url_pic, clusterNumber, directions, prep_time, cook_time, break_time,
                    difficulty,budget,serving,coordonnees)
                
        else:
             sql_recipe = """insert into recipe (id_recipe,name,categories,url_pic,clusterNumber,directions,prep_time,cook_time,
                                break_time,difficulty,budget,serving,coordonnees)\
                                values ("%s","%s","%s","%s","%s","%s","%s","%s","%s","%s","%s","%s","%s");"""  % (
                   compteur, name, categories, url_pic, clusterNumber, directions, prep_time, cook_time, break_time,
                    difficulty,budget,serving,coordonnees)
                
                
        print( "RECIPE ::", sql_recipe )
        
        # Injection des recipes en base
        try:
            # Execute the SQL command recipe
            cursor.execute(sql_recipe)
            # Commit your changes in the database
            db.commit()
        except (MySQLdb.Error, MySQLdb.Warning) as e:
                db.rollback()
                print(e)
                
        # liaison recette  recette ingredients
    compteur = 0
    for values in recipes:
        ingredients =  values['ingredients']
        title_recipe =  values['nom']
        title_recipe = title_recipe.replace("\"", "\\'")
        title_recipe = title_recipe.replace('\u0301', ' ')
        title_recipe = title_recipe.replace('\u00ae\ufe0f', ' ')
        title_recipe = str(title_recipe)
        img_url =  values['img_url']
        if ingredients is None:
            pass
            print( "0 ingrédients" )
        else:
            for ing in ingredients:
                ing_quantity = ing['quantity']
                if(ing_quantity != 'empty'):
                    ing_quantity = ''.join(ing_quantity)
                    ing_quantity = ing_quantity.replace('\u00a0', '')
                print(ing_quantity)
                ing_name = re.escape( str( ing['nom_ingre']))
                ing_name = ing_name.replace('\u0301', ' ')
                ing_name = ing_name.replace('\u00ae\ufe0f', ' ')
                ing_img_url = re.escape( str( ing['image_ingre']))
                sql_get_id_ingredient = """select id_ingredient from ingredient 
                where name= "%s";""" % (ing_name)
                print( "SQL GET ID INGRE :::::::::", sql_get_id_ingredient )
                sql_get_id_recipe = """select id_recipe from recipe where name= "%s";""" % (title_recipe)
                print( "GET ID RECIPE :", sql_get_id_recipe )
                id_fetch_rec = 0
                id_fetch = 0

                try:
                    cursor.execute(
                        """select id_recipe from recipe where name= "%s";""" % (title_recipe) )
                    id_rec = cursor.fetchall()
                    for id_rec_att in id_rec:
                        id_fetch_rec = id_rec_att[0]
                        print( "CLE PRIMAIRE RECIPE :", id_fetch_rec )
                        print("TITRE RECIPE :::::", title_recipe)
                        print("++++++++++++++++++++++++")
                except (MySQLdb.Error, MySQLdb.Warning) as e:
                        db.rollback()
                        print( "Error get ID Recipe" )
                        print(e)
                try:
                    cursor.execute(
                        """select id_ingredient from ingredient where name= "%s";""" % (ing_name) )
                    id_ingre = cursor.fetchall()
                    for id_att in id_ingre:
                        id_fetch = id_att[0]
                        print( "CLE PRIMAIRE ING = ", id_fetch )
                        print( "Title Ingredient = ", ing_name )
                except (MySQLdb.Error, MySQLdb.Warning) as e:
                    print( "Error get ID Ingredient" )
                    db.rollback()
                    print(e)
                if ing_quantity is None:
                    sql_l_recipe_ingredient = """ insert into contain_recipe_ingredient (id_recipe,id_ingredient,quantity) values("%s","%s","%s");""" % (
                    id_fetch_rec, id_fetch, None)
                    print( "LIAISON INGREDIENT : None Quantity", sql_l_recipe_ingredient )
                else:
                    sql_l_recipe_ingredient = """ insert into contain_recipe_ingredient (id_recipe,id_ingredient,quantity)  values("%s","%s","%s");""" % (
                    id_fetch_rec, id_fetch, ing_quantity)
                    print( "LIAISON INGREDIENT :", sql_l_recipe_ingredient )   
                try:
                    # Execute the SQL command liaison
                    cursor.execute( sql_l_recipe_ingredient )
                        # Commit your changes in the database
                    db.commit()
                except:
                    # Rollback in case there is any error
                    db.rollback()




In [39]:
recette = [
     
       
{"id": 1668, "nom": "Poulet au four simple et savoureux", "img_url": " https://assets.afcdn.com/recipe/20200408/109519_w1200h1200c1cx1933cy2466cxb3744cyb5616.webp 1200w", "category": ["Viande", "Viande r\u00f4tie", "Poulet r\u00f4ti"], "global_rating": "4.8", "time_prepa": "15\u00a0min", "time_repo": "-", "time_cuisson": "1\u00a0h\u00a040\u00a0min", "difficulty": "facile", "budget": "moyen", "numberP": 1, "etape": ["Pr\u00e9chauffer le four \u00e0 200\u00b0C (thermostat 6).\n Placer \u00e0 l'int\u00e9rieur du poulet deux gousses d'ail et les feuilles de laurier.", "Pr\u00e9parer la sauce en m\u00e9langeant le jus de citron et le verre de bouillon de volaille. Arroser copieusement le poulet et verser le reste du jus dans le plat.", "Saupoudrer le gros sel sur le poulet. ", "Placer le poulet dans le plat, avec les oignons coup\u00e9s en quatre et les tomates cerises. ", "Placer le poulet au four durant environ 1 heure 40 \u00e0 200\u00b0C. (Au bout de 20 min de cuisson, retourner le poulet et laisser cuire 20 min, puis le replacer \u00e0 l'endroit pour la suite de la cuisson) ", "Attention : Arroser le poulet avec son jus le plus souvent possible durant la cuisson (ajouter de l'eau si n\u00e9cessaire). Le poulet n'en sera que plus moelleux !", "R\u00e9cup\u00e9rer le jus \u00e0 la fin de la cuisson, avec les tomates et les oignons, dans un bol, et placer sur la table, avec une belle salade verte et une po\u00eal\u00e9e de pommes de terres nouvelles."], "ingredients": [{"id_ingre": 1, "nom_ingre": "oignons", "quantity": "2", "image_ingre": "https://assets.afcdn.com/recipe/20170607/67621_w96h96c1cx350cy350.jpg"}, {"id_ingre": 2, "nom_ingre": "tomates cerise", "quantity": "1", "image_ingre": "https://assets.afcdn.com/recipe/20170607/67652_w96h96c1cx350cy350.jpg"}, {"id_ingre": 3, "nom_ingre": "ail", "quantity": "3", "image_ingre": "https://assets.afcdn.com/recipe/20170607/67514_w96h96c1cx350cy350.jpg"}, {"id_ingre": 4, "nom_ingre": "jus de citron", "quantity": "1", "image_ingre": "https://assets.afcdn.com/recipe/20170607/67412_w96h96c1cx350cy350.jpg"}, {"id_ingre": 5, "nom_ingre": "bouillon de volaille", "quantity": "1", "image_ingre": "https://assets.afcdn.com/recipe/20170607/67586_w96h96c1cx350cy350.jpg"}, {"id_ingre": 6, "nom_ingre": "feuille de laurier", "quantity": "empty", "image_ingre": "https://assets.afcdn.com/recipe/20170621/68932_w96h96c1cxb700cyb700.jpg"}, {"id_ingre": 7, "nom_ingre": "sel", "quantity": "1", "image_ingre": "https://assets.afcdn.com/recipe/20170607/67687_w96h96c1cx350cy350.jpg"}, {"id_ingre": 8, "nom_ingre": "poivre", "quantity": "empty", "image_ingre": "https://assets.afcdn.com/recipe/20170607/67563_w96h96c1cx350cy350.jpg"}, {"id_ingre": 9, "nom_ingre": "poulet", "quantity": "1", "image_ingre": "https://assets.afcdn.com/recipe/20170607/67769_w96h96c1cx350cy350.jpg"}],
"cluster":0,"coordonnees":'-0.056919;-0.180640;-0.147291;-0.051992;-0.095780;0.048301;-0.092438;-0.126569;-0.055147;-0.113215;-0.058655;0.011944;-0.130409;-0.151557;-0.000568;-0.003691;-0.138078;-0.004296;0.142209;0.140012;-0.142943;-0.196644;-0.170616;-0.147056;-0.223738;0.017700;-0.110947;-0.342784;-0.012178;0.446442;-0.171837;0.107568;-0.061919;0.433266;0.016772;-0.047313;-0.039529;-0.178839;-0.031151;-0.131473;-0.017658;-0.013499;-0.013810;0.056771;-0.060243;0.027514;0.009236;-0.060978;-0.012817;-0.027842;0.012377;0.017533;-0.048152;0.015724;0.000410;0.149829;-0.018567;-0.018632;0.103305;-0.078035;0.060320;0.012101;0.021907;0.061703;-0.025470;-0.019271;-0.015757;0.004035;0.032292;-0.002565'},
{"id": 1669, "nom": "Croque-monsieur", "img_url": " https://assets.afcdn.com/recipe/20170112/28965_w1200h1200c1cx1500cy1000.webp 1200w", "category": ["Plat unique", "Croque-monsieur"], "global_rating": "4.5", "time_prepa": "10\u00a0min", "time_repo": "-", "time_cuisson": "5\u00a0min", "difficulty": "tr\u00e8s facile", "budget": "bon march\u00e9", "numberP": 1, "etape": ["Beurrez les 8 tranches de pain de mie sur une seule face. Posez 1 tranche de fromage sur chaque tranche de pain de mie. Posez 1 tranche de jambon pli\u00e9 en deux sur 4 tranches de pain de mie. Recouvrez avec les autres tartines (face non beurr\u00e9e au dessus). ", "Dans un bol m\u00e9langer le fromage r\u00e2p\u00e9 avec le lait, le sel, le poivre et la muscade. ", "R\u00e9partissez le m\u00e9lange sur les croque-monsieur. ", "Placez sur une plaque au four sous le grill pendant 10 mn."], "ingredients": [{"id_ingre": 1, "nom_ingre": "pain de mie", "quantity": "8", "image_ingre": "https://assets.afcdn.com/recipe/20170607/67589_w96h96c1cx350cy350.jpg"}, {"id_ingre": 2, "nom_ingre": "beurre tendre", "quantity": ["50", "\u00a0", "g"], "image_ingre": "https://assets.afcdn.com/recipe/20170621/68920_w96h96c1cxb300cyb300.jpg"}, {"id_ingre": 3, "nom_ingre": "gruy\u00e8re r\u00e2p\u00e9", "quantity": "100", "image_ingre": "https://assets.afcdn.com/recipe/20170607/67500_w96h96c1cx350cy350.jpg"}, {"id_ingre": 4, "nom_ingre": "lait", "quantity": ["50", "\u00a0", "g"], "image_ingre": "https://assets.afcdn.com/recipe/20170607/67388_w96h96c1cx350cy350.jpg"}, {"id_ingre": 5, "nom_ingre": "muscade", "quantity": "1", "image_ingre": "https://assets.afcdn.com/recipe/20170607/67662_w96h96c1cx350cy350.jpg"}, {"id_ingre": 6, "nom_ingre": "poivre", "quantity": "empty", "image_ingre": "https://assets.afcdn.com/recipe/20170607/67563_w96h96c1cx350cy350.jpg"}, {"id_ingre": 7, "nom_ingre": "sel", "quantity": "empty", "image_ingre": "https://assets.afcdn.com/recipe/20170607/67687_w96h96c1cx350cy350.jpg"}, {"id_ingre": 8, "nom_ingre": "jambon", "quantity": "4", "image_ingre": "https://assets.afcdn.com/recipe/20170607/67343_w96h96c1cx350cy350.jpg"}, {"id_ingre": 9, "nom_ingre": "toastinette", "quantity": "8", "image_ingre": "https://assets.afcdn.com/recipe/20210122/117531_w96h96c1cx295cy295cxb591cyb591.jpg"}],
"cluster":0,"coordonnees":'-0.056919;-0.180640;-0.147291;-0.051992;-0.095780;0.048301;-0.092438;-0.126569;-0.055147;-0.113215;-0.058655;0.011944;-0.130409;-0.151557;-0.000568;-0.003691;-0.138078;-0.004296;0.142209;0.140012;-0.142943;-0.196644;-0.170616;-0.147056;-0.223738;0.017700;-0.110947;-0.342784;-0.012178;0.446442;-0.171837;0.107568;-0.061919;0.433266;0.016772;-0.047313;-0.039529;-0.178839;-0.031151;-0.131473;-0.017658;-0.013499;-0.013810;0.056771;-0.060243;0.027514;0.009236;-0.060978;-0.012817;-0.027842;0.012377;0.017533;-0.048152;0.015724;0.000410;0.149829;-0.018567;-0.018632;0.103305;-0.078035;0.060320;0.012101;0.021907;0.061703;-0.025470;-0.019271;-0.015757;0.004035;0.032292;-0.002565'},
{"id": 1670, "nom": "R\u00f4ti de boeuf au four tout simple", "img_url": " https://assets.afcdn.com/recipe/20180621/79712_w1200h1200c1cx1944cy2592cxb3888cyb5184.webp 1200w", "category": ["Viande", "Viande r\u00f4tie", "R\u00f4ti de b\u0153uf"], "global_rating": "4.2", "time_prepa": "5\u00a0min", "time_repo": "-", "time_cuisson": "25\u00a0min", "difficulty": "facile", "budget": "bon march\u00e9", "numberP": 1, "etape": ["Pr\u00e9chauffer le four \u00e0 220\u00b0C (thermostat 7-8).\nMettre le r\u00f4ti avec barde et ficelles dans un plat qui va au four. Couper la gousse d'ail en deux et en frotter le r\u00f4ti. Parsemer le haut du r\u00f4ti de thym et l'arroser d'huile d'olive. Poser les gousses d'ail sur le r\u00f4ti.", "Enfourner pour 15 minutes de cuisson. Sortir le r\u00f4ti et retirer la barde et la ficelle. Ajouter l'eau et faire tomber l'ail dans le plat", "Saler et poivrer \u00e0 convenance le r\u00f4ti. Laisser cuire 10 \u00e0 15 minutes au four selon la cuisson d\u00e9sir\u00e9e."], "ingredients": [{"id_ingre": 1, "nom_ingre": "ail", "quantity": ["50", "\u00a0", "g"], "image_ingre": "https://assets.afcdn.com/recipe/20170607/67514_w96h96c1cx350cy350.jpg"}, {"id_ingre": 2, "nom_ingre": "huile d'olive","quantity": ["50", "\u00a0", "g"], "image_ingre": "https://assets.afcdn.com/recipe/20220114/127365_w96h96c1cx411cy411cxb822cyb822.jpg"}, {"id_ingre": 3, "nom_ingre": "eau", "quantity": "12", "image_ingre": "https://assets.afcdn.com/recipe/20190308/88958_w96h96c1cx150cy150cxb300cyb300.jpg"}, {"id_ingre": 4, "nom_ingre": "poivre", "quantity": ["50", "\u00a0", "g"], "image_ingre": "https://assets.afcdn.com/recipe/20170607/67563_w96h96c1cx350cy350.jpg"}, {"id_ingre": 5, "nom_ingre": "sel", "quantity": "1", "image_ingre": "https://assets.afcdn.com/recipe/20170607/67687_w96h96c1cx350cy350.jpg"}, {"id_ingre": 6, "nom_ingre": "thym", "quantity": "1", "image_ingre": "https://assets.afcdn.com/recipe/20170607/67735_w96h96c1cx350cy350.jpg"}, {"id_ingre": 7, "nom_ingre": "r\u00f4ti de boeuf", "quantity": "400", "image_ingre": "https://assets.afcdn.com/recipe/20180213/77542_w96h96c1cx2350cy2350cxb4700cyb4700.jpg"}],
"cluster":0,"coordonnees":'-0.056919;-0.180640;-0.147291;-0.051992;-0.095780;0.048301;-0.092438;-0.126569;-0.055147;-0.113215;-0.058655;0.011944;-0.130409;-0.151557;-0.000568;-0.003691;-0.138078;-0.004296;0.142209;0.140012;-0.142943;-0.196644;-0.170616;-0.147056;-0.223738;0.017700;-0.110947;-0.342784;-0.012178;0.446442;-0.171837;0.107568;-0.061919;0.433266;0.016772;-0.047313;-0.039529;-0.178839;-0.031151;-0.131473;-0.017658;-0.013499;-0.013810;0.056771;-0.060243;0.027514;0.009236;-0.060978;-0.012817;-0.027842;0.012377;0.017533;-0.048152;0.015724;0.000410;0.149829;-0.018567;-0.018632;0.103305;-0.078035;0.060320;0.012101;0.021907;0.061703;-0.025470;-0.019271;-0.015757;0.004035;0.032292;-0.002565'},
{"id": 1671, "nom": "Moules marini\u00e8res", "img_url": " https://assets.afcdn.com/recipe/20211214/125828_w1200h1200c1cx1309cy793cxb2121cyb1414cxt188cyt3.webp 1200w", "category": ["Fruits de mer", "Moules"], "global_rating": "4.5", "time_prepa": "45\u00a0min", "time_repo": "-", "time_cuisson": "15\u00a0min", "difficulty": "tr\u00e8s facile", "budget": "bon march\u00e9", "numberP": 1, "etape": ["Hachez les \u00e9chalotes.", "Grattez bien et lavez les moules. \nMettez-les dans une cocotte avec 1 noix de beurre, les \u00e9chalotes hach\u00e9es et le vin blanc.", "Faites-les ouvrir dans la cocotte couverte, sur feu vif pendant quelques minutes. M\u00e9langez 2 ou 3 fois pendant la cuisson.", "D\u00e8s qu'elles sont ouvertes, retirez les moules de la cocotte en conservant le jus de la cuisson. D\u00e9posez-les dans 1 plat creux et gardez-les au chaud.", "Remettez le jus sur le feu. Malaxez avec 1 fourchette 1 cuiller\u00e9e \u00e0 caf\u00e9 de farine avec le m\u00eame volume de beurre ou de margarine. Incorporez le tout au jus de la cuisson des moules sur le feu. Laissez bouillir un instant. Salez poivrez.", "Versez sur les moules. Saupoudrez de persil hach\u00e9 et servez."], "ingredients": [{"id_ingre": 1, "nom_ingre": "beurre", "quantity": ["50", "\u00a0", "g"], "image_ingre": "https://assets.afcdn.com/recipe/20201201/115998_w96h96c1cx912cy821cxb1824cyb1643.jpg"}, {"id_ingre": 2, "nom_ingre": "vin blanc sec", "quantity": ["50", "\u00a0", "g"], "image_ingre": "https://assets.afcdn.com/recipe/20170607/67422_w96h96c1cx350cy350.jpg"}, {"id_ingre": 3, "nom_ingre": "farine","quantity": ["50", "\u00a0", "g"], "image_ingre": "https://assets.afcdn.com/recipe/20170607/67682_w96h96c1cx350cy350.jpg"}, {"id_ingre": 4, "nom_ingre": "persil", "quantity": "empty", "image_ingre": "https://assets.afcdn.com/recipe/20170607/67650_w96h96c1cx350cy350.jpg"}, {"id_ingre": 5, "nom_ingre": "sel", "quantity": "empty", "image_ingre": "https://assets.afcdn.com/recipe/20170607/67687_w96h96c1cx350cy350.jpg"}, {"id_ingre": 6, "nom_ingre": "poivre", "quantity": "empty", "image_ingre": "https://assets.afcdn.com/recipe/20170607/67563_w96h96c1cx350cy350.jpg"}, {"id_ingre": 7, "nom_ingre": "moules", "quantity": "4", "image_ingre": "https://assets.afcdn.com/recipe/20170607/67458_w96h96c1cx350cy350.jpg"}, {"id_ingre": 8, "nom_ingre": "\u00e9chalotes", "quantity": "2", "image_ingre": "https://assets.afcdn.com/recipe/20170607/67402_w96h96c1cx3008cy2000.jpg"}],
"cluster":0,"coordonnees":'-0.056919;-0.180640;-0.147291;-0.051992;-0.095780;0.048301;-0.092438;-0.126569;-0.055147;-0.113215;-0.058655;0.011944;-0.130409;-0.151557;-0.000568;-0.003691;-0.138078;-0.004296;0.142209;0.140012;-0.142943;-0.196644;-0.170616;-0.147056;-0.223738;0.017700;-0.110947;-0.342784;-0.012178;0.446442;-0.171837;0.107568;-0.061919;0.433266;0.016772;-0.047313;-0.039529;-0.178839;-0.031151;-0.131473;-0.017658;-0.013499;-0.013810;0.056771;-0.060243;0.027514;0.009236;-0.060978;-0.012817;-0.027842;0.012377;0.017533;-0.048152;0.015724;0.000410;0.149829;-0.018567;-0.018632;0.103305;-0.078035;0.060320;0.012101;0.021907;0.061703;-0.025470;-0.019271;-0.015757;0.004035;0.032292;-0.002565'},

       
    
  ]
# persisetDB(recette)

In [78]:
img = 'https://assets.afcdn.com/recipe/20190220/88075_w1200h911.jpg'
# img = 'https://assets.afcdn.com/recipe/20100101/recipe_default_img_placeholder_w96h96c1.jpg'
img = img.split("/")
img5 = img[5].split("_")
if len(img5) >=4:
    img_url = img[0] + "//" + img[2] + "/" +img[3] + "/"+ img[4] + "/" + img5[0] + "_" + img5[1] + "_" + img5[2] + "_" + img5[3] + "_" + "w1000h1000.jpg"
else:
    img_url = img[0] + "//" + img[2] + "/" +img[3] + "/"+ img[4] + "/" + img5[0] + "_" + "_" + "w1000h1000.jpg"
img_url

'https://assets.afcdn.com/recipe/20190220/88075__w1000h1000.jpg'